In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets, transforms, models
from tqdm import tqdm


In [89]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [86]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [93]:
trainSet = pd.read_csv('Dataset/train.csv')

classes = trainSet[['id','species']].copy()
classes['id'] = classes['id'].astype(str)
classes['label'] = LabelEncoder().fit_transform(classes['species'])

image_folder = 'Dataset/images/'
imgs = []
labels = []
for i in sorted(os.listdir(image_folder)):
    id = i.split('.')[0]
    if id in classes['id'].values:
        labels.append(classes[classes['id'] == id]['label'].values[0])
        image = Image.open(os.path.join(image_folder, i)).convert('1')
        imgs.append(image)

print(f"There are {len(imgs)} images in the dataset")
print(f"There are {len(np.unique(labels))} labels in the dataset")


There are 990 images in the dataset
There are 99 labels in the dataset


In [94]:

X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"There are {len(X_train)} images in the training set")
print(f"There are {len(X_test)} images in the test set")
print(f"There are {len(np.unique(y_train))} classes in the training set")
print(f"There are {len(np.unique(y_test))} classes in the test set")


There are 792 images in the training set
There are 198 images in the test set
There are 99 classes in the training set
There are 99 classes in the test set


In [95]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.transform(image=np.array(self.images[index],dtype=np.float32))['image']
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label



transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2()
])

trainDataSet = CustomDataSet(images=X_train, labels=y_train, transform=transform)
testDataSet = CustomDataSet(images=X_test, labels=y_test, transform=transform)




In [106]:
batch_size = 64
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=batch_size)

min_loss_epoch = 0
min_loss_value = -1
best_model_weights_paths = {}

best_val_loss = float('inf')  # Initialize with a large value
best_epoch = -1
best_model_weights = None
train_losses = []
val_losses = []

criterion = nn.CrossEntropyLoss()
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 99)
model.to(device)
print(model)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainDataLoader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainDataLoader) / batch_size
    train_losses.append(train_loss)

    # Validation loop
    model.eval()
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in testDataLoader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    accuracy = correct_predictions / total_samples
    val_loss = 1-accuracy
    val_losses.append(val_loss)
    print(f"Epoch: {epoch+1}/{num_epochs}  Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


best_epoch = val_losses.index(min(val_losses))
min_loss_epoch = best_epoch
min_loss_value = f'{min(val_losses):.4f}'
print(f"Min Train Loss: {min(train_losses)} at Epoch {train_losses.index(min(train_losses))}  Min Val Loss: {min_loss_value[criterion]} at Epoch {best_epoch}")


# Save the best model's weights
torch.save(model, f"best_model.pth")


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Epoch 1/1000: 100%|██████████| 13/13 [00:02<00:00,  4.47batch/s]


Epoch: 1/1000  Loss: 0.0568, Val Loss: 0.8838


Epoch 2/1000: 100%|██████████| 13/13 [00:02<00:00,  4.50batch/s]


Epoch: 2/1000  Loss: 0.0250, Val Loss: 0.8081


...

Epoch 999/1000: 100%|██████████| 13/13 [00:03<00:00,  4.14batch/s]


Epoch: 999/1000  Loss: 0.0000, Val Loss: 0.0859


Epoch 1000/1000: 100%|██████████| 13/13 [00:03<00:00,  4.27batch/s]


Epoch: 1000/1000  Loss: 0.0000, Val Loss: 0.0859


TypeError: ignored